## Aktivitetsoversigt opsætning af søgedatabase
1. Installer nødvendige pakker. Skal kun gøres en gang.
2. Opret tabeller i database angivet i environment

### Installer nødvendige biblioteker
Installerer Python pakker. Skal kun gøres en gang.

Husk også at sætte environment variable op. De er:
- Database navn (POSTGRES_DB)
- Database bruger (POSTGRES_USER)
- Database pasord (POSTGRES_PASSWORD)

In [ ]:
### Installer pakker
%pip install psycopg2-binary
%pip install python_dotenv

### Opret tabeller til chunket tekst i Postgresql 
3 tabeller:
1. Books: Metaoplysninger om PDF url, bogtitel og antal sider.
2. Pages: Råt udtræk af siderne i alle bøger
3. Chunks: Vektor tabellen med tekststykker og tilhørende embeddings

In [3]:
import psycopg2
from pgvector.psycopg2 import register_vector
from dotenv import load_dotenv
import os

load_dotenv()
database = os.getenv("POSTGRES_DB", None)
db_user = os.getenv("POSTGRES_USER", None)
db_password = os.getenv("POSTGRES_PASSWORD", None)

cn = psycopg2.connect(
    host="localhost",
    database=database,
    user=db_user,
    password=db_password,
)

cur = cn.cursor()

cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
register_vector(cn)
# cur.execute("DROP TABLE IF EXISTS chunks")
# cur.execute("DROP TABLE IF EXISTS pages")
# cur.execute("DROP TABLE IF EXISTS books")

# cur.execute("CREATE TABLE IF NOT EXISTS chunks_udentitel \
#             (id BIGSERIAL PRIMARY KEY, \
#              book_id integer NOT NULL REFERENCES books ON DELETE RESTRICT, \
#              sidenr int NOT NULL, \
#              chunk text NOT NULL, \
#              embedding VECTOR(1536))")

cur.execute("CREATE TABLE IF NOT EXISTS books \
            (id SERIAL PRIMARY KEY, \
             pdf_navn text NOT NULL,  \
             titel text NOT NULL,  \
             forfatter text,  \
             antal_sider int NOT NULL, \
             created_datetime TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP)")

cur.execute("CREATE TABLE IF NOT EXISTS pages \
            (id SERIAL PRIMARY KEY, \
             book_id integer NOT NULL REFERENCES books ON DELETE RESTRICT, \
             sidenr int NOT NULL, \
             antal_ord int NOT NULL, \
             tekst text NOT NULL)")


cur.execute("CREATE TABLE IF NOT EXISTS chunks \
            (id BIGSERIAL PRIMARY KEY, \
             book_id integer NOT NULL REFERENCES books ON DELETE RESTRICT, \
             sidenr int NOT NULL, \
             chunk text NOT NULL, \
             embedding VECTOR(1536))")

# cur.execute("CREATE TABLE IF NOT EXISTS chunks_large \
#             (id BIGSERIAL PRIMARY KEY, \
#              book_id integer NOT NULL REFERENCES books ON DELETE RESTRICT, \
#              sidenr int NOT NULL, \
#              chunk text NOT NULL, \
#              embedding VECTOR(1536))")


cn.commit()

cur.close()
cn.close()

### Opdater _books_ med dato for oprettelse

In [ ]:
import psycopg2
from dotenv import load_dotenv
import os

load_dotenv()
database = os.getenv("POSTGRES_DB", None)
db_user = os.getenv("POSTGRES_USER", None)
db_password = os.getenv("POSTGRES_PASSWORD", None)

cn = psycopg2.connect(
    host="localhost",
    database=database,
    user=db_user,
    password=db_password
)

cur = cn.cursor()

cur.execute("DELETE FROM chunks")

cur.execute("ALTER TABLE books ADD COLUMN IF NOT EXISTS created_datetime TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP")

cn.commit()

### Tøm tabeller efter behov
Hjælpe rutine til at tømme tabellerne

In [1]:

import psycopg2
from dotenv import load_dotenv
import os

load_dotenv()
database = os.getenv("POSTGRES_DB", None)
db_user = os.getenv("POSTGRES_USER", None)
db_password = os.getenv("POSTGRES_PASSWORD", None)

cn = psycopg2.connect(
    host="localhost",
    database=database,
    user=db_user,
    password=db_password
)

cur = cn.cursor()

cur.execute("DELETE FROM chunks")
# cur.execute("DELETE FROM test_chunks")
# cur.execute("DELETE FROM test_spm")
# cur.execute("DELETE FROM pages")
# cur.execute("DELETE FROM books")

cn.commit()

cur.close()
cn.close()

## Indlæs bøger i lokale tabeller
Books og Chunks tabellerne fyldes af det egentlige oprettelsesprogram (læs_pdf_filer.py). Denne rutine indlæser teksten i Pages, så det bliver nemmere at forstå chunking og embedding problemer.

In [ ]:
import psycopg2
import requests
import pymupdf
from dotenv import load_dotenv
import os

load_dotenv()
database = os.getenv("POSTGRES_DB", None)
db_user = os.getenv("POSTGRES_USER", None)
db_password = os.getenv("POSTGRES_PASSWORD", None)

def save_book(book) -> None:
    cn = psycopg2.connect(
    host="localhost",
    database=database,
    user=db_user,
    password=db_password
    )

    cur = cn.cursor()

    
    cur.execute(
        "INSERT INTO books(pdf_navn, titel, forfatter, antal_sider) "
        + "VALUES (%s, %s, %s, %s) RETURNING id",
        (book["pdf-url"], book["titel"], book["forfatter"], book["antal_sider"]),
    )

    book_id = cur.fetchone()[0]
 
    print(f"{book["pdf-url"]} -> book_id: {book_id} {book["titel"]}")

    for page in book["sider"]:
        cur.execute("INSERT INTO pages(book_id, sidenr, antal_ord, tekst) VALUES (%s, %s, %s, %s)",
                    (book_id, page["side"], page["antalord"], page["tekst"]))
        
    cn.commit()
    cur.close()
    cn.close()


with open("samlet_input.txt", "rb") as pdfer:
    for url in filter(lambda x: not x[0] == "#", pdfer):
        url = url.strip()

        try:
            r = requests.get(url)
            r.raise_for_status()
        except requests.exceptions.HTTPError as e:
            print(f"HTTP-fejl opstod: {e}")
            continue
        except requests.exceptions.ConnectionError as e:
            print(f"Forbindelsesfejl opstod: {e}")
            continue
        except requests.exceptions.Timeout as e:
            print(f"Timeout-fejl opstod: {e}")
            continue
        except requests.exceptions.RequestException as e:
            print(f"En ukendt fejl opstod: {e}")
            continue

        pdf = pymupdf.open("pdf", r.content)

        metadata = pdf.metadata
        book = {
            "pdf-url": str(url.decode("utf-8")),  # pdf_url,
            "titel": metadata["title"],
            "forfatter": metadata["author"],
            "antal_sider": len(pdf),
            "sider": [],
        }
        
        for i in range(1, len(pdf) - 1) :
            text = pdf[i].get_text("text")
            antalord = len(text.split())
            page = { "side": i, "antalord": antalord, "tekst": text }
            book["sider"].append(page)

        pdf.close()

        save_book(book)

### Fjern anetavler (fra PoC)

In [5]:
import psycopg2
from dotenv import load_dotenv
import os

anetavle_urler = [
"https://slaegtsbibliotek.dk/2024/904285.pdf",
"https://slaegtsbibliotek.dk/2024/904286.pdf",
"https://slaegtsbibliotek.dk/2024/904288.pdf",
"https://slaegtsbibliotek.dk/2024/904289.pdf",
"https://slaegtsbibliotek.dk/2024/904291.pdf",
"https://slaegtsbibliotek.dk/2024/904293.pdf",
"https://slaegtsbibliotek.dk/2024/904295.pdf",
"https://slaegtsbibliotek.dk/2024/904298.pdf",
"https://slaegtsbibliotek.dk/2024/904319.pdf",
"https://slaegtsbibliotek.dk/2024/904321.pdf",
"https://slaegtsbibliotek.dk/2024/904323.pdf",
"https://slaegtsbibliotek.dk/2024/904325.pdf",
"https://slaegtsbibliotek.dk/2024/904327.pdf",
"https://slaegtsbibliotek.dk/2024/904330.pdf",
"https://slaegtsbibliotek.dk/2024/904332.pdf",
"https://slaegtsbibliotek.dk/2024/904356.pdf",
"https://slaegtsbibliotek.dk/2024/904381.pdf",
"https://slaegtsbibliotek.dk/2024/904404.pdf",
"https://slaegtsbibliotek.dk/2024/904405.pdf",
"https://slaegtsbibliotek.dk/2024/904406.pdf"]

load_dotenv()
database = os.getenv("POSTGRES_DB", None)
db_user = os.getenv("POSTGRES_USER", None)
db_password = os.getenv("POSTGRES_PASSWORD", None)

cn = psycopg2.connect(
    host="localhost",
    database=database,
    user=db_user,
    password=db_password
)

cur = cn.cursor()

for url in anetavle_urler:
    cur.execute("select id from books where pdf_navn = %s", (url,))
    bookid = cur.fetchone()

    if bookid is None:
        print(f"Book not found: {url}")
        continue
    else:
        cur.execute("delete from test_spm where book_id = %s", (bookid[0],))
        cur.execute("delete from test_chunks where book_id = %s", (bookid[0],))
        cur.execute("delete from chunks_large where book_id = %s", (bookid[0],))
        cur.execute("delete from chunks_tiny where book_id = %s", (bookid[0],))
        cur.execute("delete from chunks_small where book_id = %s", (bookid[0],))
        cur.execute("delete from chunks_udentitel where book_id = %s", (bookid[0],))
        cur.execute("delete from chunks where book_id = %s", (bookid[0],))
        cur.execute("delete from pages where book_id = %s", (bookid[0],))
        cur.execute("delete from books where id = %s", (bookid[0],))
        print(f"Book deleted: {url}")

cn.commit()

cur.close()
cn.close()

Book deleted: https://slaegtsbibliotek.dk/2024/904285.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904286.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904288.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904289.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904291.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904293.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904295.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904298.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904319.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904321.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904323.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904325.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904327.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904330.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904332.pdf
Book deleted: https://slaegtsbibliotek.dk/2024/904356.pdf
Book not found: https://slaegtsbibliotek.dk/2024/904381.pdf
Book deleted